In [15]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import csv

def get_url(position, location):
    base_url ='https://pk.indeed.com/jobs?q={}&l={}'
    url=base_url.format(position, location)
    return url

def get_record(card):
    """Extract job data from a single record"""
    atag = card.h2.a.span
    job_title=atag.get('title')
    company_title = card.find("span",{"class":"companyName"}).text.strip()
    post_date = card.find('span', 'date').text
    today = datetime.today().strftime('%Y-%m-%d')
    summary = card.find('div', 'job-snippet').text.strip().replace('\n', ' ')
 # this does not exists for all jobs, so handle the exceptions
    salary_tag = card.find('div', 'attribute_snippet')
    if salary_tag:
        salary = salary_tag.text.strip()
    else:
        salary = ''  
    job_url = 'https://www.indeed.com' + card.h2.a.get('href')
    record = (job_title, company_title, post_date, today, summary, salary, job_url)
    return record

def main(position, location):
    """Run the main program routine"""
    records = []
    urll = get_url(position, location)

    while True:
        response = requests.get(urll)
        soup = BeautifulSoup(response.text, 'html.parser')
        cards = soup.find_all('div', 'job_seen_beacon')
        for card in cards:
            record = get_record(card)
            records.append(record) 
        try:
            url = 'https://www.indeed.com' + soup.find('a', {'aria-label': 'Next'}).get('href')
        except AttributeError:
            break
#     info_all_df3=pd.DataFrame.from_dict(record,orient='index',columns=['JobTitle', 'Company', 'PostDate', 'ExtractDate', 'Summary', 'Salary', 'JobUrl'])
#     info_all_df3.to_csv('indeedjobspython.csv')                                   
    with open('resultsjobs.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['JobTitle', 'Company', 'PostDate', 'ExtractDate', 'Summary', 'Salary', 'JobUrl'])
        writer.writerows(records)

In [16]:
main('python', 'Lahore')